# 1. Introduction

- Task
  - Kaggle의 Bag of Words Meets Bags of Popcorn이라는 감성분석 튜토리얼을 진행
  - 참고: https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview

- Information
  - 주어지는 데이터셋은 train set과 test set 모두 25,000개의 긍정 부정 리뷰로 구성되어 있으며, 긍정인 경우엔 label이 1이고, 부정인 경우엔 label이 0.

# 2. Install packages & Load Files


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords

# NLTK의 stopwords 다운로드
nltk.download('stopwords')

# PorterStemmer를 임포트
from nltk.stem import PorterStemmer

# NLTK의 wordnet 리소스 다운로드
nltk.download('wordnet')

# 추가 wordnet 리소스 다운로드
nltk.download('omw-1.4')

# WordNetLemmatizer를 임포트
from nltk.stem import WordNetLemmatizer

# Matplotlib 라이브러리 임포트
import matplotlib as plt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)
test = pd.read_csv('testData.tsv', header=0, delimiter='\t', quoting=3)
submit = pd.read_csv('sampleSubmission.csv')

# 3. EDA

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [ ]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      25000 non-null  object
 1   review  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


In [ ]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [ ]:
submit.head(30)

,id,sentiment
0,12311_10,0
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,0
5,2913_8,0
6,4396_1,0
7,395_2,0
8,10616_1,0
9,9074_9,0


In [ ]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


In [ ]:
# 긍정과 부정 라벨 개수 동일
train['sentiment'].value_counts()

sentiment
1    12500
0    12500
Name: count, dtype: int64

In [ ]:
submit['sentiment'].value_counts()

sentiment
0    25000
Name: count, dtype: int64

#4. Text Preprocessing

## 4-1. HTML 태그 제거

In [ ]:
train['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [ ]:
from bs4 import BeautifulSoup

위 리뷰를 soup 객체로 바꿔주면, html 태그를 제외하고 text만 온전히 추출하는 작업이 가능합니다. 이 기능을 해주는 것이 `get_text()`입니다. 첫번째 리뷰를 하나의 예시로 들어봅시다. 윗문장이 제거 전, 아래문장이 제거 후입니다.


In [ ]:
# BeautifulSoup을 사용하여 HTML 태그 제거
example1 = BeautifulSoup(train["review"][0])


In [ ]:
example1

<html><body><p>"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br/><br/>Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br/><br/>The actual feature film bit when it 

In [ ]:
# HTML 태그를 제거하고 텍스트만 추출
example1 = example1.get_text()

# 태그 제거 전 텍스트 출력
print("before deleting:", train['review'][0])

# 태그 제거 후 텍스트 출력
print("after deleting:", example1)

before deleting: "With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit wh

## 4-2. 정규표현식(re)으로 알파벳만 남기기

- html 태그는 제거했지만, 각종 특수문자가 아직도 남아있다. 알파벳만 남기기 위해 정규표현식(regular expression)을 사용.


### **Q1) 아래 letters_only를 통해 example1에서 알파벳을 제외한 나머지 글자를 알파벳만 남길수 있도록 공백으로 대체하세요**

In [ ]:
import re

# HTML 태그가 제거된 텍스트를 대상으로 알파벳 외 모든 문자 제거
# [^a-zA-Z]+는 알파벳이 아닌 문자들이 하나 이상 연속으로 나타나는 모든 부분을 찾는 패턴
# re.sub(pattern, repl, string): 문자열 string에서 pattern에 매칭되는 모든 부분을 repl로 대체
letters_only = re.sub(r'[^a-zA-Z]+', ' ', example1)

print(letters_only)


 With all this stuff going down at the moment with MJ i ve started listening to his music watching the odd documentary here and there watched The Wiz and watched Moonwalker again Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent Moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for minutes or s

## 4-3. 토큰화(Tokenizing)

- 토큰(token)은 의미를 갖는 최소 분석 단위
- 토큰화(tokenizing) 는 corpus 덩어리를 작은 토큰 단위로 쪼개어주는 작업
- 영어에서는 대체로 띄어쓰기를 기준으로 토큰화를 진행
- `split()` 함수를 쓰면 띄어쓰기(단어) 단위로 토큰화가 진행될 것. 아래 예시를 통해 하나의 문장에서 437개의 단어로 토큰화된 것을 확인할 수 있다.
- `lower_case` 함수로 소문자로 변환(대문자와 소문자는 다른 단어로 구분되기에 이를 그대로 두면 복잡성을 야기할 수 있기에)
- 물론 무작정 소문자로 통합하는 것도 지양. 미국을 뜻하는 US와 우리를 뜻하는 us는 엄밀히 다르기 때문.

In [ ]:
# letters_only 문자열을 모두 소문자로 변환
lower_case = letters_only.lower()

# 소문자로 변환된 문자열을 공백 기준으로 단어별로 분리 (토큰화)
token_words = lower_case.split()

# 토큰화된 단어의 개수를 출력
print("토큰화 이후 생성된 토큰(단어) 개수: ", len(token_words))


토큰화 이후 생성된 토큰(단어) 개수:  437


참고)
- nltk와 케라스에서 토큰화를 수행해주는 도구들을 제공
- 이 도구들은 아포스트로피가 들어간 단어들을 토큰화
- 본 과제에서는 앞서 정규표현식으로 아포스트로피를 모두 제거했기에 굳이 아래 도구들을 사용하지 않고 `split()`로만 토큰화를 수행

In [ ]:
# from nltk.tokenize import word_tokenize
# word_tokenize(lower_case)
# from nltk.tokenize import WordPunctTokenizer
# WordPunctTokenizer().tokenize(lower_case)
# from tensorflow.keras.preprocessing.text import text_to_word_sequence
# text_to_word_sequence(lower_case)

## 4-4. 불용어(Stopword) 제거

- 단어들 중에서 무의미한 것들을 제거하는 작업이 필요
- ex. I, we, you, our 등 굉장히 자주 등장하지만 분석에서 크게 중요하지 않은 단어
- `nltk`에서는 영어의 불용어들을 제공

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

# NLTK에 내장된 불용어 리스트 가져오기
stopwords_list = stopwords.words('english')

# 불용어 리스트의 총 개수 출력
print("nltk에 내장된 불용어 개수: ", len(stopwords_list))

# 불용어 리스트에서 첫 10개 단어 출력
print("불용어 예시: ", stopwords_list[:10])

nltk에 내장된 불용어 개수:  179
불용어 예시:  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [ ]:
# 불용어 제거 후 남은 단어들 리스트 생성
non_stopwords = [w for w in token_words if not w in stopwords_list]

# 불용어 제거 후 남은 단어의 총 개수 출력
print("예시 review에서 불용어 제거하고 남은 토큰 개수: ", len(non_stopwords))

예시 review에서 불용어 제거하고 남은 토큰 개수:  219


## 4-5. 어간 추출(Stemming)

- nltk에선 단어에서 어간을 추출해주는 도구를 제공
- 하지만 이 어간 추출의 작업은 정확히 어간만 추출하는 것이 아니라 정해진 규칙에 따라 임의로 어미를 떼어놓는 작업이기에 100% 정확하게 어간만 추출하지는 못한다.

- nltk의 포터 알고리즘의 규칙은 다음과 같다.
  - ALIZE → AL
  - ANCE → 제거
  - ICAL → IC

- 예시를 들어보면, 아래와 같은 어간 추출을 진행
  - formalize → formal
  - allowance → allow
  - electricical → electric
  - 출처: 딥러닝을 이용한 자연어 처리 입문 위키독스(https://wikidocs.net/21707)

In [ ]:
from nltk.stem import PorterStemmer

# PorterStemmer 객체 생성
porter_stemmer = PorterStemmer()

# 불용어가 제거된 단어 리스트를 스테밍 처리
stemmed_words = [porter_stemmer.stem(w) for w in non_stopwords]

# 스테밍 처리된 단어 리스트 출력 (필요 시)
print(stemmed_words[:10])  # 첫 10개의 스테밍 처리된 단어 출력


['stuff', 'go', 'moment', 'mj', 'start', 'listen', 'music', 'watch', 'odd', 'documentari']


## 4-6. 표제어 추출(Lemmatization)

- 표제어 추출도 어간 추출과 마찬가지의 역할을 수행한다. 이 둘은 본질적인 차이를 갖는데, Stemming은 단어 그 자체만을 고려하지만 Lemmatization은 그 단어가 문장 속에서 어떤 품사(Part-of-speech)로 쓰였는지까지 판단한다는 점이다.
- 따라서 Lemmatization이 보다 복잡한 처리 과정을 거칠 수밖에 없다.

In [ ]:
# NLTK의 wordnet 및 omw-1.4 데이터 다운로드
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# WordNetLemmatizer 객체 생성
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
lemmatized_words = [wordnet_lemmatizer.lemmatize(w) for w in non_stopwords]
lemmatized_words = [wordnet_lemmatizer.lemmatize(w,"v") for w in lemmatized_words]

In [ ]:
print(non_stopwords[21:40]) # none
print(stemmed_words[21:40]) # stemming
print(lemmatized_words[21:40]) # lemmatization

['really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema']
['realli', 'cool', 'eighti', 'mayb', 'make', 'mind', 'whether', 'guilti', 'innoc', 'moonwalk', 'part', 'biographi', 'part', 'featur', 'film', 'rememb', 'go', 'see', 'cinema']
['really', 'cool', 'eighty', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'go', 'see', 'cinema']


## 4-7. 하나의 함수로 표현

앞선 6가지의 전처리 과정을 하나의 함수로 통합

In [ ]:
stopwords_list = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

def review_to_words(raw_review):
  except_tag = BeautifulSoup(raw_review).get_text() # html 태그 제거
  letters_only = re.sub("[^a-zA-Z]", " ", except_tag) # 알파벳만 남기기
  token_words = letters_only.lower().split() # 소문자로 통합하고 단어 단위로 토큰화
  non_stopwords = [w for w in token_words if not w in stopwords_list] # 불용어 삭제
  lemmatized_words = [wordnet_lemmatizer.lemmatize(w) for w in non_stopwords]
  lemmatized_words = [wordnet_lemmatizer.lemmatize(w,"v") for w in lemmatized_words] # Lemmatization
  words = " ".join(lemmatized_words)
  return words

# 5. BoW(Bag of Words) 형태로 변환

- Bag of words는 단어의 출현 빈도에 집중하여 텍스트를 수치화하는 표현 방식

- 예를 들어 아래처럼 두 개의 노래 구절이 있는 경우

```python
lyric1 = "but I don't want to stay in the middle"
lyric2 = "like you a little don't want no riddle"
```

- 아래 같은 단어 모음이 만들어지고

```python
vocab = ["but", "I", "don't", "want", "to", "stay", "in", "the", "middle", "like", "you", "a", "little", "no", "riddle"]
```

- 아래처럼 단어들을 빈도로 수치화 가능

```python
lyric1 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
lyric2 = [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
```

위 같은 작업을 사이킷런의 `CountVectorizer`가 수행

In [ ]:
# 전체 리뷰 개수
num_reviews = train['review'].size

# 정제된 리뷰 리스트 초기화
clean_train_reviews = []

# 각 리뷰에 대해 정제 과정 적용
for i in range(0, num_reviews):
    if (i + 1) % 5000 == 0:  # 실행이 잘되는지 확인하기 위해 5000개 실행될 때마다 확인 문구 출력
        print('Review {} of {}'.format(i + 1, num_reviews))
    clean_train_reviews.append(review_to_words(train['review'][i]))

# 결과 확인
print("첫 번째 정제된 리뷰: ", clean_train_reviews[0])

<ipython-input-43-4620c86a40d2>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  except_tag = BeautifulSoup(raw_review).get_text() # html 태그 제거


Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
Review 25000 of 25000
첫 번째 정제된 리뷰:  stuff go moment mj start listen music watch odd documentary watch wiz watch moonwalker maybe want get certain insight guy think really cool eighty maybe make mind whether guilty innocent moonwalker part biography part feature film remember go see cinema originally release subtle message mj feel towards press also obvious message drug bad kay visually impressive course michael jackson unless remotely like mj anyway go hate find bore may call mj egotist consent make movie mj fan would say make fan true really nice actual feature film bite finally start minute exclude smooth criminal sequence joe pesci convince psychopathic powerful drug lord want mj dead bad beyond mj overhear plan nah joe pesci character rant want people know supply drug etc dunno maybe hate mj music lot cool thing like mj turn car robot whole speed demon sequence also director must patience saint c

`clean_train_reviews`에는 25,000개의 전처리된 리뷰들이 담겨있습니다. 이것을 `CountVectorizer에` input으로 넣어주면 BoW가 완성됩니다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 객체 생성
vectorizer = CountVectorizer(
    analyzer='word',         # 단어 단위로 분석
    tokenizer=None,          # 별도의 토크나이저 사용하지 않음
    preprocessor=None,       # 별도의 전처리기 사용하지 않음
    stop_words=None,         # 불용어 직접 지정하지 않음
    min_df=2,                # 토큰이 나타날 최소 문서 개수
    ngram_range=(1, 2),      # 단어의 묶음 개수 (1-그램과 2-그램)
    max_features=4000        # 최대 토큰 개수 (컬럼의 최대 개수)
)


# 6. TfidfVectorizer

- `TidfVectorizer()`는 TF-IDF(단어빈도*문서빈도역수)를 학습시키는 함수.
- 단어빈도란 특정 단어가 한 문서 내에서 출현한 빈도이고, 문서빈도는 특정 단어가 출현한 전체 문서의 개수.
- 왜 두 빈도를 모두 알아야 할까? 예컨대 정관사 a, the는 영어에서 굉장히 많이 쓰인다. 하지만 출현빈도에 비해 그렇게 중요치 않은 단어이다. 이처럼 단어빈도는 높아도 중요도가 낮은 단어들을 걸러주기 위해 이 단어가 출현한 문서의 빈도도 카운트해줄 필요가 있다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer(
    min_df=4,                # 토큰이 나타날 최소 문서 개수
    analyzer='word',         # 학습 단위
    ngram_range=(1, 2),      # 단어의 묶음 개수 (1-그램과 2-그램)
    max_features=1000        # 최대 토큰 개수 (컬럼의 최대 개수)
)


In [ ]:
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features.shape

(25000, 1000)

In [ ]:
# 피처 이름(단어) 가져오기
vocab = vectorizer.get_feature_names_out()

# 피처 이름 일부 출력
print(vocab[:10])

['ability' 'able' 'absolutely' 'accent' 'accept' 'across' 'act' 'action'
 'actor' 'actress']


# 7. Modeling


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Feature와 target 데이터 설정
x = train_data_features
y = train['sentiment']

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

# 트리 알고리즘 3개를 사용
rf = RandomForestClassifier(n_estimators=200,
                            n_jobs=-1,
                            random_state=42,
                            max_depth=20)

xgb = XGBClassifier(n_estimators=200,
                    max_depth=10,
                    learning_rate=0.05,
                    objective='binary:logistic',
                    use_label_encoder=False,  # For XGBoost 1.3.0 and later
                    eval_metric='logloss')    # For XGBoost 1.3.0 and later

lgbm = LGBMClassifier(n_estimators=200,
                      max_depth=10,
                      metric='binary_logloss')



In [ ]:
xgb.fit(x_train, y_train)
y_pred_xgb = xgb.predict_proba(x_val)

In [ ]:
lgbm.fit(x_train, y_train)
y_pred_lgbm = lgbm.predict_proba(x_val)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 8738, number of negative: 8762
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.202809 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 197823
[LightGBM] [Info] Number of data points in the train set: 17500, number of used features: 1000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499314 -> initscore=-0.002743
[LightGBM] [Info] Start training from score -0.002743
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [ ]:
rf.fit(x_train, y_train)
y_pred_rf = rf.predict_proba(x_val)

In [ ]:
print('Random Forest AUC Score :', roc_auc_score(y_val, y_pred_rf[:,1]))
print('XGBoost AUC Score :', roc_auc_score(y_val, y_pred_xgb[:,1]))
print('LGBM AUC Score :', roc_auc_score(y_val, y_pred_lgbm[:,1]))

Random Forest AUC Score : 0.9076715878903935
XGBoost AUC Score : 0.9164858292593361
LGBM AUC Score : 0.9262710316820311


# 8. test set 추론

In [ ]:
num_reviews = test['review'].size

clean_test_reviews = []
for i in range(0, num_reviews):
     if (i + 1) % 5000 == 0 :  #실행이 잘 되는지 확인하기 위해 5000개 실행될때마다 확인문구
         print('Review {} of {}'.format(i+1, num_reviews))
     clean_test_reviews.append(review_to_words(test['review'][i]))

<ipython-input-43-4620c86a40d2>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  except_tag = BeautifulSoup(raw_review).get_text() # html 태그 제거


Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
Review 25000 of 25000


In [ ]:
test_data_features = vectorizer.fit_transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [ ]:
# 3개 알고리즘 중 원하는 것으로 predict
result = lgbm.predict(test_data_features)
submit['sentiment'] = result

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [ ]:
submit.head(10)

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,1
3,7186_2,1
4,12128_7,0
5,2913_8,0
6,4396_1,0
7,395_2,1
8,10616_1,0
9,9074_9,0
